# OpenFlights Datasets

In [19]:
# If running in a fresh environment, uncomment the next line to install pandas
# %pip install -q pandas requests

import pandas as pd
from io import StringIO
import requests

AIRPORTS_URL = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
AIRLINES_URL = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat"
ROUTES_URL = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

# Explicit schemas based on OpenFlights documentation
airport_columns = [
    "airport_id", "name", "city", "country", "iata", "icao",
    "latitude", "longitude", "altitude", "timezone", "dst",
    "tz_database_time_zone", "type", "source"
]

airline_columns = [
    "airline_id", "name", "alias", "iata", "icao", "callsign",
    "country", "active"
]

route_columns = [
    "airline", "airline_id", "source_airport", "source_airport_id",
    "destination_airport", "destination_airport_id", "codeshare",
    "stops", "equipment"
]


def fetch_csv(url: str) -> str:
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    return response.text


def load_openflights_table(url: str, columns: list[str]) -> pd.DataFrame:
    raw_text = fetch_csv(url)
    # OpenFlights files are comma-separated but may include quoted fields
    df = pd.read_csv(StringIO(raw_text), header=None, names=columns)
    return df


airports_df = load_openflights_table(AIRPORTS_URL, airport_columns)
airlines_df = load_openflights_table(AIRLINES_URL, airline_columns)
routes_df = load_openflights_table(ROUTES_URL, route_columns)

# Basic type coercions for numeric-like columns
for col in ["airport_id", "altitude", "timezone"]:
    if col in airports_df.columns:
        airports_df[col] = pd.to_numeric(airports_df[col], errors="coerce")

for col in ["latitude", "longitude"]:
    if col in airports_df.columns:
        airports_df[col] = pd.to_numeric(airports_df[col], errors="coerce")

for col in ["airline_id"]:
    if col in airlines_df.columns:
        airlines_df[col] = pd.to_numeric(airlines_df[col], errors="coerce")

for col in ["stops"]:
    if col in routes_df.columns:
        routes_df[col] = pd.to_numeric(routes_df[col], errors="coerce")

airports.dat

Airport ID, Name, City, Country, IATA, ICAO,
Latitude, Longitude, Altitude, Timezone, DST,
Tz database time zone, Type, Source

In [20]:
airports_df

,airport_id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz_database_time_zone,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,NaN,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,NaN,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,NaN,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,NaN,\N,\N,airport,OurAirports


airlines.dat

Airline ID, Name, Alias, IATA, ICAO, Callsign, Country, Active

In [21]:
airlines_df

,airline_id,name,alias,iata,icao,callsign,country,active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N
...,...,...,...,...,...,...,...,...
6157,21248,GX Airlines,NaN,NaN,CBG,SPRAY,China,Y
6158,21251,Lynx Aviation (L3/SSX),NaN,NaN,SSX,Shasta,United States,N
6159,21268,Jetgo Australia,NaN,JG,\N,NaN,Australia,Y
6160,21270,Air Carnival,NaN,2S,\N,NaN,India,Y


routes.dat

Airline, Airline ID, Source airport, Source airport ID,
Destination airport, Destination airport ID, Codeshare,
Stops, Equipment

In [22]:
routes_df

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [23]:
# Export dataframes to CSV files
from datetime import datetime
import os

# Ensure export directory exists
os.makedirs('cleaned_dataset', exist_ok=True)

# Create timestamp for filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Export each dataframe to separate CSV files
airports_df.to_csv('raw_dataset/airports.csv', index=False, encoding='utf-8')
airlines_df.to_csv('raw_dataset/airlines.csv', index=False, encoding='utf-8')
routes_df.to_csv('raw_dataset/routes.csv', index=False, encoding='utf-8')


# Data Cleaning 

In [24]:
# FINAL DATA CLEANING PROCESS
# Create cleaned datasets from original data

# Airports cleaning
airports_cleaned = airports_df.copy()

# 1. Better handling of string values
airports_cleaned = airports_cleaned.replace({
    '\\N': pd.NA,
    'nan': pd.NA,
    'NaN': pd.NA,
    '': pd.NA,
    'Unknown': pd.NA,
    'unknown': pd.NA
})

# 2. Clean IATA and ICAO codes
airports_cleaned['iata'] = airports_cleaned['iata'].replace(['-', 'nan', 'NaN'], pd.NA)
airports_cleaned['icao'] = airports_cleaned['icao'].replace(['-', 'nan', 'NaN'], pd.NA)

# 3. Coordinate validation
airports_cleaned = airports_cleaned.dropna(subset=['latitude', 'longitude'])
airports_cleaned = airports_cleaned[
    (airports_cleaned['latitude'] >= -90) & (airports_cleaned['latitude'] <= 90) &
    (airports_cleaned['longitude'] >= -180) & (airports_cleaned['longitude'] <= 180)
]

# 4. Clean other columns
airports_cleaned['altitude'] = pd.to_numeric(airports_cleaned['altitude'], errors='coerce').fillna(0)
airports_cleaned['timezone'] = pd.to_numeric(airports_cleaned['timezone'], errors='coerce')
dst_mapping = {'E': 'E', 'A': 'A', 'S': 'S', 'O': 'O', 'Z': 'Z', 'N': 'N', 'U': 'U'}
airports_cleaned['dst'] = airports_cleaned['dst'].map(dst_mapping).fillna('U')
airports_cleaned['type'] = airports_cleaned['type'].fillna('airport')
airports_cleaned['source'] = airports_cleaned['source'].fillna('Unknown')
airports_cleaned = airports_cleaned.drop_duplicates(subset=['airport_id'], keep='first')

# Clean string columns
string_columns = ['name', 'city', 'country', 'iata', 'icao', 'tz_database_time_zone', 'type', 'source']
for col in string_columns:
    if col in airports_cleaned.columns:
        airports_cleaned[col] = airports_cleaned[col].astype(str).str.strip()
        airports_cleaned[col] = airports_cleaned[col].replace('nan', pd.NA)

print("Airports cleaned successfully!")
print(f"Airports: {len(airports_cleaned)} rows")


Airports cleaned successfully!
Airports: 7698 rows


In [25]:
# Airlines cleaning
airlines_cleaned = airlines_df.copy()

# 1. Better handling of string values
airlines_cleaned = airlines_cleaned.replace({
    '\\N': pd.NA,
    'nan': pd.NA,
    'NaN': pd.NA,
    '': pd.NA,
    'Unknown': pd.NA,
    'unknown': pd.NA,
    '-': pd.NA
})

# 2. Clean airline_id - remove invalid IDs
airlines_cleaned['airline_id'] = pd.to_numeric(airlines_cleaned['airline_id'], errors='coerce')
airlines_cleaned = airlines_cleaned[airlines_cleaned['airline_id'] > 0]

# 3. Clean IATA and ICAO codes
airlines_cleaned['iata'] = airlines_cleaned['iata'].replace(['-', 'nan', 'NaN'], pd.NA)
airlines_cleaned['icao'] = airlines_cleaned['icao'].replace(['-', 'nan', 'NaN'], pd.NA)

# 4. Remove airlines without any valid codes
airlines_cleaned = airlines_cleaned[
    ~(airlines_cleaned['iata'].isna() & airlines_cleaned['icao'].isna())
]

# 5. Clean other columns
airlines_cleaned['callsign'] = airlines_cleaned['callsign'].replace(['-', 'nan', 'NaN'], pd.NA)
airlines_cleaned['active'] = airlines_cleaned['active'].replace(['\\N', 'nan', 'NaN'], 'N').fillna('N')
airlines_cleaned['country'] = airlines_cleaned['country'].fillna('Unknown')
airlines_cleaned = airlines_cleaned.drop_duplicates(subset=['airline_id'], keep='first')

# Clean string columns
string_columns = ['name', 'alias', 'iata', 'icao', 'callsign', 'country']
for col in string_columns:
    if col in airlines_cleaned.columns:
        airlines_cleaned[col] = airlines_cleaned[col].astype(str).str.strip()
        airlines_cleaned[col] = airlines_cleaned[col].replace('nan', pd.NA)

print("Airlines cleaned successfully!")
print(f"Airlines: {len(airlines_cleaned)} rows")


Airlines cleaned successfully!
Airlines: 6159 rows


In [26]:
# Routes cleaning
routes_cleaned = routes_df.copy()

# 1. Better handling of string values
routes_cleaned = routes_cleaned.replace({
    '\\N': pd.NA,
    'nan': pd.NA,
    'NaN': pd.NA,
    '': pd.NA,
    'Unknown': pd.NA,
    'unknown': pd.NA,
    '-': pd.NA
})

# 2. Clean airline_id - remove invalid IDs
routes_cleaned['airline_id'] = pd.to_numeric(routes_cleaned['airline_id'], errors='coerce')
routes_cleaned = routes_cleaned[routes_cleaned['airline_id'] > 0]

# 3. Clean airport IDs
routes_cleaned['source_airport_id'] = pd.to_numeric(routes_cleaned['source_airport_id'], errors='coerce')
routes_cleaned['destination_airport_id'] = pd.to_numeric(routes_cleaned['destination_airport_id'], errors='coerce')

# 4. Remove routes with missing critical information
routes_cleaned = routes_cleaned.dropna(subset=['source_airport', 'destination_airport'])
routes_cleaned = routes_cleaned.dropna(subset=['source_airport_id', 'destination_airport_id'])

# 5. Clean other columns
routes_cleaned['stops'] = pd.to_numeric(routes_cleaned['stops'], errors='coerce').fillna(0)
routes_cleaned['codeshare'] = routes_cleaned['codeshare'].fillna('N')
routes_cleaned['equipment'] = routes_cleaned['equipment'].fillna('Unknown')

# 6. Remove routes where source and destination are the same
routes_cleaned = routes_cleaned[
    routes_cleaned['source_airport'] != routes_cleaned['destination_airport']
]

# 7. Remove duplicate routes
routes_cleaned = routes_cleaned.drop_duplicates(
    subset=['airline_id', 'source_airport_id', 'destination_airport_id'], 
    keep='first'
)

# 8. Validate references
valid_airline_ids = set(airlines_cleaned['airline_id'].dropna())
valid_airport_ids = set(airports_cleaned['airport_id'].dropna())
routes_cleaned = routes_cleaned[routes_cleaned['airline_id'].isin(valid_airline_ids)]
routes_cleaned = routes_cleaned[routes_cleaned['source_airport_id'].isin(valid_airport_ids)]
routes_cleaned = routes_cleaned[routes_cleaned['destination_airport_id'].isin(valid_airport_ids)]

# Clean string columns
string_columns = ['airline', 'source_airport', 'destination_airport', 'codeshare', 'equipment']
for col in string_columns:
    if col in routes_cleaned.columns:
        routes_cleaned[col] = routes_cleaned[col].astype(str).str.strip()
        routes_cleaned[col] = routes_cleaned[col].replace('nan', pd.NA)

print("Routes cleaned successfully!")
print(f"Routes: {len(routes_cleaned)} rows")


Routes cleaned successfully!
Routes: 66315 rows


In [27]:
# EXPORT FINAL CLEANED DATASETS
from datetime import datetime
import os

# Ensure export directory exists
os.makedirs('cleaned_dataset', exist_ok=True)

# Create timestamp for filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Export final cleaned dataframes to separate CSV files
airports_cleaned.to_csv(f'cleaned_dataset/airports_cleaned.csv', index=False, encoding='utf-8')
airlines_cleaned.to_csv(f'cleaned_dataset/airlines_cleaned.csv', index=False, encoding='utf-8')
routes_cleaned.to_csv(f'cleaned_dataset/routes_cleaned.csv', index=False, encoding='utf-8')

print("=== FINAL CLEANED DATASETS EXPORTED ===")
print("Files created in 'cleaned_dataset':")
print(f"- airports_cleaned_final_{timestamp}.csv")
print(f"- airlines_cleaned_final_{timestamp}.csv")
print(f"- routes_cleaned_final_{timestamp}.csv")

print("\nFinal dataset sizes:")
print(f"Airports: {len(airports_cleaned)} rows")
print(f"Airlines: {len(airlines_cleaned)} rows")
print(f"Routes: {len(routes_cleaned)} rows")


=== FINAL CLEANED DATASETS EXPORTED ===
Files created in 'cleaned_dataset':
- airports_cleaned_final_20251015_194444.csv
- airlines_cleaned_final_20251015_194444.csv
- routes_cleaned_final_20251015_194444.csv

Final dataset sizes:
Airports: 7698 rows
Airlines: 6159 rows
Routes: 66315 rows
